In [ ]:
'''
This version of code contains:
*Window detection with detecron2 (model 7, threshold=0.5)
*Removing overlapped window bboxes
*getting middle 1/3 of image
* Window counting from the number of bbox in vertical cuts
* getting middle vertical part 1/3 of image
→ Pixel based bbox window distance statistics
'''

'\nThis version of code contains:\n*Window detection with detecron2 (model 7, threshold=0.5)\n*Removing overlapped window bboxes\n*getting middle 1/3 of image\n* Window counting from the number of bbox in vertical cuts\n* getting middle vertical part 1/3 of image\n→ Pixel based bbox window distance control (window filtering)\n'

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.
#exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 8.9 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=68a8881e47687bdc309d9313bfc4b522d861754c55b5df7c2a0a320fa79f8a7a
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 1.6 MB/s 
     |████████████████████████████████| 147 kB 8.5 MB/s 
     |████████████████████████████████| 50 kB 4.1 MB/s 
     |████████████████████████████████| 130 kB 58.8 MB/s 
     |████████████████████████████████| 74 kB 4.2 MB/s 
     |████████████████████████████████| 749 kB 69.6 MB/s 
     |████████████████████████████████

In [ ]:
import os
import re
import json
import math
import itertools
import cv2 as cv
import numpy as np
import pandas as pd
from statistics import mean, pstdev
from collections import Counter
from collections import namedtuple
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import AffinityPropagation

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2 import model_zoo
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




##**Removing overlapped window bboxes**


In [ ]:
def get_horizontally_cropped_img(img_path):
  #This function crops the given picture so that the middle 1/3 of the picture remains.
  img = cv.imread(img_path)
  rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  h=img.shape[0]
  w=img.shape[1]
  crop_size=int(w/3)
  cropped_img = rgb[:, crop_size:w-crop_size]
  return cropped_img

In [ ]:
def get_cropped_img(img_path):
  #This function crops the given picture so that the middle 1/3 of the picture remains.
  img = cv.imread(img_path)
  rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  h=img.shape[0]
  w=img.shape[1]
  crop_size_x=int(w/3)
  crop_size_y=int(h/12)
  cropped_img_x = rgb[:, crop_size_x:w-crop_size_x]
  cropped_img_y = cropped_img_x[4*crop_size_y:9*crop_size_y,:]
  return cropped_img_y

In [ ]:
def make_prediction(im,model_pth):
  start = time.time()
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
  cfg.MODEL.WEIGHTS = model_pth # path for final model
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 #class number
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
  predictor = DefaultPredictor(cfg)
  #im = cv.imread(img_path)
  outputs = predictor(im)
  v = Visualizer(im[:, :, :],
                #metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
                scale=1,
                instance_mode=ColorMode.IMAGE)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  #img=cv.cvtColor(out.get_image(), cv.COLOR_BGR2RGB)
  img=out.get_image()
  #plt.imshow(img)
  end = time.time()
  print("Prediction time:",end - start)
  #x1, y1, x2, y2
  bbox_coordinates=outputs['instances'].pred_boxes.tensor.cpu().numpy()
  
  
  return bbox_coordinates ,img

In [ ]:
def draw_bbox_img(rgb,bbox_coordinates):
  
  canvas = np.copy(rgb) * 0  # creating a blank to draw lines on

  [cv.rectangle(canvas, (i[0],i[1]), (i[2],i[3]), (255,255,255), 5) for i in bbox_coordinates]

  #plt.imshow(canvas)
  return canvas

In [ ]:
def get_img_names(image_folder_directory):
  # assign directory
  directory = image_folder_directory
  image_paths=[]
  # iterate over files in
  # that directory
  for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      image_paths.append(str(f))
  return image_paths

In [ ]:
def get_realfloornumber(building_dataframe,image_name):
  
  floorinfo = building_dataframe.loc[building_dataframe['FileName']==image_name]
  if floorinfo.empty:
    print('Can not find real floor number')
    return -1
  else:
    realfloornumber = floorinfo['Floor'].tolist()[0]
    return realfloornumber

In [ ]:
def area(a, b):  # returns None if rectangles don't intersect
    #find intersection area of rectangles a and b
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy

In [ ]:
def remove_overlapped_bboxes(bboxes):
  #This func removes smaller bbox if it's more than 30% overlaps with a bigger bbox
  Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')
  remove_list=[]
  #print("bboxes", type(bboxes),bboxes)
  filtered_bboxes=bboxes.tolist()
  #filtered_bboxes=bboxes
  #print("filtered_bboxes",filtered_bboxes)
  for bbox_a, bbox_b in itertools.combinations(bboxes, 2):
    a = Rectangle(bbox_a[0],bbox_a[1],bbox_a[2],bbox_a[3])
    b = Rectangle(bbox_b[0],bbox_b[1],bbox_b[2],bbox_b[3])
    intersection_area=area(a, b)
    if intersection_area is not None:
      area_a = area(a, a)
      area_b = area(b, b)
      if area_a > area_b and intersection_area > ( area_b*0.3):
        #filtered_bboxes = np.delete(filtered_bboxes, np.argwhere(filtered_bboxes == bbox_b))     
        #print("bbox_b",bbox_b)   
        try:
          filtered_bboxes.remove([bbox_b[0],bbox_b[1],bbox_b[2],bbox_b[3]])
        except ValueError:
            pass
      elif area_a < area_b and intersection_area > ( area_a*0.3):
        #filtered_bboxes = np.delete(filtered_bboxes, np.argwhere(filtered_bboxes == bbox_a))
        #filtered_bboxes.remove(bbox_a)
        try:
          filtered_bboxes.remove([bbox_a[0],bbox_a[1],bbox_a[2],bbox_a[3]])
        except ValueError:
            pass
  '''
  print("remove_list",remove_list)  
  for bbox_rec in remove_list:
    bbox=[bbox_rec[0],bbox_rec[1],bbox_rec[2],bbox_rec[3]]
    #[filtered_bboxes.drop(x) for x in bboxes if x.tolist() == bbox]
    print("type bboxes", type(bboxes))
    x = [x for x in bboxes if x.tolist() == bbox]
    filtered_bboxes = np.delete(filtered_bboxes, np.argwhere(filtered_bboxes == x))

  if len(remove_list)==0:
    filtered_bboxes=bboxes
  '''
  #print("len bbox", len(bboxes), "len filtered_bboxes", len(filtered_bboxes))
  #print("filtered_bboxes",filtered_bboxes)
  return filtered_bboxes

In [ ]:
def get_intersection_coordinate(a, b):  # returns coordinates of intersection rectangle
    #find intersection coordinates of rectangles a and b
    xmax = min(a.xmax, b.xmax) 
    xmin = max(a.xmin, b.xmin)
    ymax = min(a.ymax, b.ymax) 
    ymin = max(a.ymin, b.ymin)
    coordinate = (xmin, ymin ,xmax, ymax)
    return coordinate

In [ ]:
def get_bboxes_in_roi(bbox_coordinates, roi_coordinates):
  Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')
  roi_rectangle = Rectangle(roi_coordinates[0],roi_coordinates[1],roi_coordinates[2],roi_coordinates[3])
  bboxes_in_roi=[]
  filtered_bbox_coordinates = remove_overlapped_bboxes(bbox_coordinates)
  for bbox in filtered_bbox_coordinates:
    bbox_rectangle = Rectangle(bbox[0],bbox[1],bbox[2],bbox[3])
    intersection = area(bbox_rectangle, roi_rectangle) #type change might be required
    if intersection is not None:
      bbox_area = area(bbox_rectangle,bbox_rectangle)
      if intersection >= bbox_area/2:        
        bboxes_in_roi.append(get_intersection_coordinate(bbox_rectangle, roi_rectangle))
  return bboxes_in_roi

In [ ]:
def get_bboxes_in_cut(bbox_coordinates, roi_coordinates):
  Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')
  roi_rectangle = Rectangle(roi_coordinates[0],roi_coordinates[1],roi_coordinates[2],roi_coordinates[3])
  bboxes_in_cut=[]
  #filtered_bbox_coordinates = remove_overlapped_bboxes(bbox_coordinates)
  #for bbox in filtered_bbox_coordinates:
  for bbox in bbox_coordinates:
    bbox_rectangle = Rectangle(bbox[0],bbox[1],bbox[2],bbox[3])
    intersection = area(bbox_rectangle, roi_rectangle) #type change might be required
    if intersection is not None:
      bbox_area = area(bbox_rectangle,bbox_rectangle)
      bboxes_in_cut.append(get_intersection_coordinate(bbox_rectangle, roi_rectangle))
  return bboxes_in_cut

In [ ]:
def get_horizontally_cropped_bbox_coordinates(img, cropped_img, bbox_coordinates):
  h=img.shape[0]
  w=img.shape[1]
  w_cropped=cropped_img.shape[1]
  coordinate_shift= (w - w_cropped)/2
  roi_coordinates = (coordinate_shift, 0 , coordinate_shift*2, h) #x1, y1, x2, y2
  cropped_bbox_coordinates = get_bboxes_in_roi(bbox_coordinates, roi_coordinates)
  shifted_bbox_coordinates=[]
  for bbox in cropped_bbox_coordinates:
    x1=int(bbox[0]-coordinate_shift)
    x2=int(bbox[2]-coordinate_shift)
    y1=int(bbox[1])
    y2=int(bbox[3])
    shifted_bbox_coordinates.append((x1,y1,x2,y2))
  return shifted_bbox_coordinates

In [ ]:
def get_cropped_bbox_coordinates(img, cropped_img, bbox_coordinates):
  h=img.shape[0]
  w=img.shape[1]
  w_cropped=cropped_img.shape[1]
  h_cropped=cropped_img.shape[0]
  coordinate_shift_x= (w - w_cropped)/2
  coordinate_shift_y= h/12
  roi_coordinates = (coordinate_shift_x, 4*coordinate_shift_y , coordinate_shift_x*2, 9*coordinate_shift_y) #x1, y1, x2, y2
  cropped_bbox_coordinates = get_bboxes_in_roi(bbox_coordinates, roi_coordinates)
  shifted_bbox_coordinates=[]
  for bbox in cropped_bbox_coordinates:
    x1=int(bbox[0]-coordinate_shift_x)
    x2=int(bbox[2]-coordinate_shift_x)
    y1=int(bbox[1]-4*coordinate_shift_y)
    y2=int(bbox[3]-4*coordinate_shift_y)
    shifted_bbox_coordinates.append((x1,y1,x2,y2))
  return shifted_bbox_coordinates

In [ ]:
def get_bbox_count_in_vertical_cuts(cropped_img,bbox_coordinates,cut_width=30):
  img_height=cropped_img.shape[0]
  img_width=cropped_img.shape[1]
  xstart=0
  xend=cut_width
  cut_list=[] #xstart, ystart, xend, yend , cut coordinates
  cut_count= int(img_width/cut_width)
  for i in range(cut_count):
    cut_list.append([xstart,0,xend,img_height])
    xstart=xstart+cut_width
    xend=xend+cut_width

  floor_count_list=[] # bbox_count for each vertical cut
  bbox_coordinates_in_cut_list = [] # bbox coordinates for each vertical cut
  for cut in cut_list:
    #print("get_bboxes_in_roi",get_bboxes_in_roi(bbox_coordinates, cut))
    bbox_coordinates_in_cut = get_bboxes_in_cut(bbox_coordinates, cut)
    bbox_count_in_cut = len(bbox_coordinates_in_cut)
    #print("bbox_count_in_cut",bbox_count_in_cut)
    floor_count_list.append(bbox_count_in_cut) 
    bbox_coordinates_in_cut_list.append(bbox_coordinates_in_cut)
          
  return floor_count_list, cut_list, bbox_coordinates_in_cut_list

In [ ]:
def get_floor_counts(floor_count_list):
  filtered_floor_count_list = [i for i in floor_count_list if i != 0]
  if len(filtered_floor_count_list) > 0:
    #print('Filtered Floor Count List= ', filtered_floor_count_list)
    average_floor = mean(filtered_floor_count_list)
    c = Counter(filtered_floor_count_list)  
    mode_floor = c.most_common(1)
    min_floor = min(filtered_floor_count_list)
    max_floor = max(filtered_floor_count_list)

    max_value_cut_index = [i for i, j in enumerate(floor_count_list) if j == max_floor]

  else: 
    average_floor = -1 
    mode_floor = -1
    min_floor = -1
    max_floor = -1
    max_value_cut_index = [i for i, j in enumerate(floor_count_list) if j == max_floor]


  return average_floor, mode_floor, min_floor, max_floor, max_value_cut_index

In [ ]:
def draw_floor_count_img(img, realfloornumber, average_floor, mode_floor, min_floor, max_floor):
  #img = cv.imread(img_path)
  height = img.shape[0]
  width = img.shape[1]

  canvas = 255 * np.ones([height,width,3], np.uint8)
  font = cv.FONT_HERSHEY_SIMPLEX

  text1="Floor Counts "
  start_pt1=(100,int(height/4))
  cv.putText(canvas, text1, start_pt1, font, 3, (0, 0, 0), 5, cv.LINE_AA)

  text2="Avg :"+str(average_floor)
  start_pt2=(start_pt1[0], start_pt1[1]+150)
  cv.putText(canvas, text2, start_pt2, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  text3="Mode : "+str(mode_floor)
  start_pt3=(start_pt2[0], start_pt2[1]+150)
  cv.putText(canvas, text3, start_pt3, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  text4=str("Min : " + str(min_floor))
  start_pt4=(start_pt3[0], start_pt3[1]+150)
  cv.putText(canvas, text4, start_pt4, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  text5=str("Max : " + str(max_floor))
  start_pt5=(start_pt4[0], start_pt4[1]+150)
  cv.putText(canvas, text5, start_pt5, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  return canvas


In [ ]:
def draw_concat_img(image_name, save_folder, pred_img, cropped_img, cropped_bbox_coordinates, realfloornumber, average_floor, mode_floor, min_floor, max_floor, cut_list_both_cropped, average_distance, mode_distance, min_distance, max_distance, std_deviation):
  img_save_name= "concatenated"+image_name
  h_original=pred_img.shape[0]
  h_cropped = cropped_img.shape[0]

  cut_shift= int(h_original/3)

  #Bboxes
  bbox_img=draw_bbox_img(cropped_img, cropped_bbox_coordinates)
  #Vertical Cut Img
  vertical_cut_img = np.copy(cropped_img) 
  [cv.rectangle(vertical_cut_img, (i[0],i[1]+cut_shift), (i[2],i[3]+cut_shift), (255,102,0), 5) for i in cut_list_both_cropped]
  #Floor count result IMG
  floor_count_img = draw_floor_count_img(cropped_img, realfloornumber, average_floor, mode_floor, min_floor, max_floor)
  #Floor statistics IMG
  bbox_statistics_img = draw_bbox_distance_statistics_img(cropped_img, average_distance, mode_distance, min_distance, max_distance, std_deviation)

  im_list = [pred_img, bbox_img, vertical_cut_img, floor_count_img,bbox_statistics_img]
  #concatenated_img = cv.hconcat([resized, bbox_img, vertical_cut_img, floor_count_img])
  concatenated_img = hconcat_resize_min(im_list)

  cv.imwrite(save_folder+"/"+img_save_name, cv.cvtColor(concatenated_img, cv.COLOR_BGR2RGB))

In [ ]:
def hconcat_resize_min(im_list, interpolation=cv.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation) for im in im_list]
    return cv.hconcat(im_list_resize)


##**Bbox distance pattern**

In [ ]:
def pixel_based_window_distance(bbox_coordinates_in_cut_list, max_value_cut_index):
  #max_value_cut_index is index of cuts that contains max number of bboxes
  y1_differences_in_cuts=[]
  for i in max_value_cut_index:
    sorted_coordinates = sorted(bbox_coordinates_in_cut_list[i], key=lambda x: x[1]) #sort bbox coordinates in given cut based on y1 value
    y1_differences = [sorted_coordinates[i+1][1]-sorted_coordinates[i][1] for i in range(len(sorted_coordinates)-1)] #get difference of two consecutive element of list based on y1
    y1_differences_in_cuts.append(y1_differences)  
  
  total_y1_differences_in_cuts = [item for sublist in y1_differences_in_cuts for item in sublist]
  return total_y1_differences_in_cuts


In [ ]:
def get_bbox_distance_statistics(y1_differences):
  if len(y1_differences) > 0:
    #print('Filtered Floor Count List= ', filtered_floor_count_list)
    average_distance= mean(y1_differences)
    c = Counter(y1_differences)  
    mode_distance = c.most_common(1)
    min_distance = min(y1_differences)
    max_distance = max(y1_differences)
    std_deviation = pstdev(y1_differences)
  else: 
    average_distance = -1 
    mode_distance = -1
    min_distance = -1
    max_distance = -1
    std_deviation = -1

  return average_distance, mode_distance, min_distance, max_distance, std_deviation

In [ ]:
def draw_bbox_distance_statistics_img(img,average_distance, mode_distance, min_distance, max_distance, std_deviation):
  #img = cv.imread(img_path)
  height = img.shape[0]
  width = img.shape[1]

  canvas = 255 * np.ones([height,width,3], np.uint8)
  font = cv.FONT_HERSHEY_SIMPLEX

  text0="Bbox Dist. Stats."
  start_pt0=(100,int(height/4))
  cv.putText(canvas, text0, start_pt0, font, 3, (0, 0, 0), 5, cv.LINE_AA)

  text1="Std dv. : "+str(std_deviation)
  start_pt1=(start_pt0[0], start_pt0[1]+150)
  cv.putText(canvas, text1, start_pt1, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  text2="Avg :"+str(average_distance)
  start_pt2=(start_pt1[0], start_pt1[1]+150)
  cv.putText(canvas, text2, start_pt2, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  text3="Mode : "+str(mode_distance)
  start_pt3=(start_pt2[0], start_pt2[1]+150)
  cv.putText(canvas, text3, start_pt3, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  text4=str("Min : " + str(min_distance))
  start_pt4=(start_pt3[0], start_pt3[1]+150)
  cv.putText(canvas, text4, start_pt4, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  text5=str("Max : " + str(max_distance))
  start_pt5=(start_pt4[0], start_pt4[1]+150)
  cv.putText(canvas, text5, start_pt5, font, 3, (0, 0, 0), 3, cv.LINE_AA)

  return canvas

In [ ]:
def vertical_line_approach(model_pth, img_path, building_dataframe, save_folder):  
  img = cv.imread(img_path)
  img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  image_name = img_path.split('/')[-1]
  realfloornumber = get_realfloornumber(building_dataframe,image_name)

  bbox_coordinates , pred_img = make_prediction(img, model_pth)

  cropped_img = get_cropped_img(img_path) #crop both horizontally and vertically for bbox distance statistics
  cropped_bbox_coordinates = get_cropped_bbox_coordinates(img, cropped_img, bbox_coordinates)

  horizontally_cropped_img = get_horizontally_cropped_img(img_path) #crop just vertically for floor count
  horizontally_cropped_bbox_coordinates = get_horizontally_cropped_bbox_coordinates(img, horizontally_cropped_img, bbox_coordinates)

  floor_count_list, cut_list, bbox_coordinates_in_cut_list = get_bbox_count_in_vertical_cuts(horizontally_cropped_img,horizontally_cropped_bbox_coordinates,cut_width=30)
  floor_count_list_both_cropped, cut_list_both_cropped, bbox_coordinates_in_cut_list_both_cropped = get_bbox_count_in_vertical_cuts(cropped_img,cropped_bbox_coordinates,cut_width=30)
  average_floor, mode_floor, min_floor, max_floor, max_value_cut_index = get_floor_counts(floor_count_list)
  average_floor_both_cropped, mode_floor_both_cropped, min_floor_both_cropped, max_floor_both_cropped, max_value_cut_index_both_cropped = get_floor_counts(floor_count_list_both_cropped)
  y1_differences = pixel_based_window_distance(bbox_coordinates_in_cut_list_both_cropped, max_value_cut_index_both_cropped)
  average_distance, mode_distance, min_distance, max_distance, std_deviation = get_bbox_distance_statistics(y1_differences)
  draw_concat_img(image_name, save_folder, pred_img, horizontally_cropped_img, horizontally_cropped_bbox_coordinates,realfloornumber, average_floor, mode_floor, min_floor, max_floor, cut_list_both_cropped,average_distance, mode_distance, min_distance, max_distance, std_deviation)
  

In [ ]:
save_folder="/content/drive/MyDrive/şehringözü-Depo/etiketliveri/WindowCounting/Results/Vertical_Cut_Results/20.04.2022_Bbox_distance_v2"
building_dataframe_pth="/content/drive/MyDrive/şehringözü-Depo/etiketliveri/WindowCounting/until20220224_BuildingFloorCount.xlsx"
image_folder="/content/drive/MyDrive/şehringözü-Depo/etiketliveri/WindowCounting/Images/Uskudar/clean_data_20220405"
model_pth="/content/drive/MyDrive/şehringözü-Depo/etiketliveri/Window/Output/Model7-It3000lr00025_0001/model_final.pth"

point_results=[]
line_results=[]
img_paths = get_img_names(image_folder)
building_dataframe = pd.read_excel(building_dataframe_pth)
for img_path in img_paths:
  vertical_line_approach(model_pth, img_path, building_dataframe, save_folder)

Can not find real floor number


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


Prediction time: 2.1645092964172363
Can not find real floor number
Prediction time: 2.1014885902404785
Can not find real floor number
Prediction time: 2.0150396823883057
Can not find real floor number
Prediction time: 2.0694329738616943
Can not find real floor number
Prediction time: 2.0030457973480225
Can not find real floor number
Prediction time: 2.12319278717041
Can not find real floor number
Prediction time: 1.965383768081665
Can not find real floor number
Prediction time: 2.0608694553375244
Can not find real floor number
Prediction time: 2.041271209716797
Can not find real floor number
Prediction time: 1.9866669178009033
Can not find real floor number
Prediction time: 2.0000503063201904
Can not find real floor number
Prediction time: 2.1295371055603027
Can not find real floor number
Prediction time: 2.0190794467926025
Can not find real floor number
Prediction time: 2.0720832347869873
Can not find real floor number
Prediction time: 2.0348198413848877
Can not find real floor number